In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import math
import quandl
import collections, functools, operator
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import itemgetter
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

In [33]:
def data_cleaning(data):
    '''
    @author: Wuding Li
    @The function cleans the data
    '''
    
    data = data[data['date'] != 19251231]
    #counts = data['date'].value_counts().to_dict()
    
    #adjust date format
    data['date'] = data['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    
    
    #remove PRC and SHROUT missing. According to the describtion, PRC missing meanings totally missing without a clue
    data['PRC'] = pd.to_numeric(data.PRC, errors='coerce')
    data = data.dropna(subset = ['PRC'])
    data['PRC'] = data['PRC'].abs()
    
    data['SHROUT'] = pd.to_numeric(data.SHROUT, errors='coerce')
    data = data.dropna(subset = ['SHROUT'])
    
    #get the medium
    NYSE = pd.read_excel('NYSE medium.xlsx')
    NYSE['date'] = pd.to_datetime(NYSE['date'])
    data['YearMonth'] = data['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE['YearMonth'] = NYSE['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE = NYSE.drop(columns=['date'])    
    data = pd.merge(data,NYSE,how = 'left',on = 'YearMonth')
    data = data[data['YearMonth'] != '192512']
    #counts = data['date'].value_counts().to_dict()
    

    data['mkt_cap'] = data['PRC']*data['SHROUT']
    data = data[data['mkt_cap']*1000>data['market cap']] #SHROUT is approximate to 10**3

    
    #Remove ADR CLosed end fund and foreign fund
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.ffill())
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.bfill())
    data = data[data['SHRCD'].isin([11.0,10.0])]
    
    
    #check more than 5
    mask = (data['PRC'] > 5)
    data = data.loc[mask]

    #clean the RET and RETX column. 'C' stands for the first trading month. Then we change it to 0
    data['RET'] = data['RET'].apply(pd.to_numeric, errors='coerce').fillna(0)
    data['RETX'] =data['RETX'].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    data = data.drop(columns=['YearMonth'])


    return data

In [34]:
def rolling_time_window(Start_Month, Start_Year, Time_Period_Training, Time_Period_Testing, Gap, data):
    '''
    @author: Wuding Li
    @The function returns two cleaned datasets. One is the sampling period data and the other is testing period data
    @variables: Start_Month: the starting month of our rolling time period
                Start_Year: the starting year of our rolling time period
                Time_Period_Training: Span of sampling Period
                Time_Period_Testing: Span of testing Period
                Gap: Gap between sampling and testing
                Data: The input data dataset
    @Also this function apply the filter no closed end fund and stock price more than 5 dollars
    
    @Updated: 10/7 fixed the end_date_test "frequency = 0" problem
    @Updated: 10/16 change "DATE" to "date" to match the updated format
    @Updated: 11/1 used the French's medium to substitute our formal medium which directly get from dataset
    '''
    
    #get the start and end date of data
    start_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=1, freq='D')[0].date())
    end_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=2, freq=str(Time_Period_Training-1)+'M')[-1].date())
    mask_train = (data['date'] >= start_date_train) & (data['date'] <= end_date_train)
    
    
    start_month_testing = Start_Month+Time_Period_Training+Gap
    if start_month_testing % 12 == 0:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)-1
        start_month_testing = 12
    else:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)
        start_month_testing = start_month_testing % 12    
    start_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq='D')[0].date())
    end_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq=str(Time_Period_Testing)+'M')[-1].date())
    mask_test = (data['date'] >= start_date_test) & (data['date'] <= end_date_test)
    
    #applying filter functions below on the master dataset
    mask_master = (data['date'] >= start_date_train) & (data['date'] <= end_date_test)
    data_master = data.loc[mask_master]
    

    full = []
    for i in data_master['PERMNO'].values:
        if (data_master[data_master['PERMNO'] == i]).shape[0] == (Time_Period_Training+ Time_Period_Testing+ Gap):
            full.append(i)
    data_master = data_master[data_master['PERMNO'].isin(full)]

    
    return data_master.loc[mask_train],data_master.loc[mask_test]


In [35]:
def format_FF():
    '''
    @author: Zhikang Wang
    @The function read and clean up the fama_french dataset pulled from Dr. French's website
    @Variable: input is fama_french dataset pulled from Dr. French's website
    '''
    
    ff_data = pd.read_csv("fama_french.csv")
    # Getting and renaming the columns we need
    ff_ret = ff_data[['YearMonth','Mkt-RF','SMB','HML','RF']]
    ff_ret = ff_ret.rename(columns = {"Mkt-RF": "mkt_ret"})
    # converting the factor exposures to percentage
    ff_ret["mkt_ret"] = ff_ret["mkt_ret"]/100
    ff_ret["SMB"] = ff_ret["SMB"]/100
    ff_ret["HML"] = ff_ret["HML"]/100
    ff_ret["RF"] = ff_ret["RF"]/100
    ff_ret["YearMonth"] = ff_ret["YearMonth"].astype(str) #converting YearMonth to string
    
    return ff_ret

In [36]:
def get_weight(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['weight']=data.groupby(['date'])['RET'].apply(lambda x: (x-x.mean())*(-1/count))
    data['weight'] = data.groupby(['PERMNO'])['weight'].shift(1)
    data = data.dropna(subset = ['weight'])
    return data

In [37]:
def get_weight_Residual(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the residual's weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    #data['residual_weight']=data.groupby(['YearMonth'])['residual'].apply(lambda x: (x-x.mean())*(-1/count)).shift(1)
    data['residual_weight']=data.groupby(['YearMonth'])['residual'].apply(lambda x: (x-x.mean())*(-1/count))
    data['residual_weight'] = data.groupby(['PERMNO'])['residual_weight'].shift(1)
    data = data.dropna(subset = ['residual_weight'])
    return data

In [38]:
def get_equal_weight(data):
    '''
    @author: Zhikang Wang
    @The function add a column to the data set that assigned equal weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['1'] = [1]*len(data)
    data['equal_weight']=data.groupby(['date'])['1'].apply(lambda x: x*(1/count)).shift(1)
    data = data.dropna()
    return data

In [39]:
def format_Grundy(df_conditional):
    '''
    @author: Robin Lam, Zhikang Wang
    This function formats the dataframe which is used in the conditional factor model in the spirit of Grundy.
        if excess returns on the RMRF, SMB, HML factors in month t are positive, use that factor;
        otherwise, factor = 0.
    Input:
        df_conditional: dataframe acquired after running the regression model for each stock (TOTAL)
            variables: the ones from output.csv
    Output:
        df_conditional: same dataframe as the input, but added the following: mkt_ret_UP, SMB_UP, HML_UP
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP
    '''
    df_conditional = df_conditional[['YearMonth','PERMNO','mkt_ret','SMB','HML','RF',
                                    'ticker_ret','ticker_excess_ret','weight',
                                    'mkt_factor','SMB_factor','HML_factor']]
    
    df_conditional['mkt_factor'] = df_conditional.groupby(['PERMNO'])['mkt_factor'].apply(lambda x: x.shift(1))
    df_conditional['SMB_factor'] = df_conditional.groupby(['PERMNO'])['SMB_factor'].apply(lambda x: x.shift(1))
    df_conditional['HML_factor'] = df_conditional.groupby(['PERMNO'])['HML_factor'].apply(lambda x: x.shift(1))
    df_conditional = df_conditional.dropna()
    df_conditional.loc[df_conditional['mkt_factor'] > 0, 'mkt_ret_UP'] = df_conditional['mkt_ret']    # this month, t
    df_conditional.loc[df_conditional['mkt_factor'] <= 0, 'mkt_ret_UP'] = 0 
    df_conditional.loc[df_conditional['SMB_factor'] > 0, 'SMB_UP'] = df_conditional['SMB']    # this month, t
    df_conditional.loc[df_conditional['SMB_factor'] <= 0, 'SMB_UP'] = 0
    df_conditional.loc[df_conditional['HML_factor'] > 0, 'HML_UP'] = df_conditional['HML']    # this month, t
    df_conditional.loc[df_conditional['HML_factor'] <= 0, 'HML_UP'] = 0
    df_conditional = df_conditional.drop(columns=['mkt_factor','SMB_factor','HML_factor'])
    
    return df_conditional

In [40]:
def regression_model(combined_temp):
    '''
    @author: Robin Lam, Yulin Chen, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        combined_temp: dataframe which contains returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        combined_temp: same dataframe as the input, but with an extra column (residual)
    
    @Update: 10/16/2019: Wuding Li Change the output date to match the paper 
    '''
    x = combined_temp[['mkt_ret','SMB','HML']]
    y = combined_temp['ticker_excess_ret']
    
    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared
    
    # calculating the residuals for each regression model
    combined_temp['residual'] = y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] - coef[0]
    combined_temp['mkt_factor'] = x['mkt_ret']*coef[1]    #used in conditiona_factor_model
    combined_temp['SMB_factor'] = x['SMB']*coef[2]    #used in conditiona_factor_model
    combined_temp['HML_factor'] = x['HML']*coef[3]    #used in conditiona_factor_model
    
    # adding coefficients to columns
    combined_temp['alpha'] = coef[0]
    combined_temp['mkt_beta'] = coef[1]
    combined_temp['SMB_beta'] = coef[2]
    combined_temp['HML_beta'] = coef[3]
    combined_temp['r_squared'] = r_squared
    
    # adding t-statistics to columns
    combined_temp['t_alpha'] = t_test[0]
    combined_temp['t_mkt_beta'] = t_test[1]
    combined_temp['t_SMB_beta'] = t_test[2]
    combined_temp['t_HML_beta'] = t_test[3]
    
    
    combined_temp['port_mkt_beta'] = combined_temp['mkt_beta']*combined_temp['weight']
    combined_temp['port_SMB_beta'] = combined_temp['SMB_beta']*combined_temp['weight']
    combined_temp['port_HML_beta'] = combined_temp['HML_beta']*combined_temp['weight']
    
    standard_dev = combined_temp['residual'].std() #get the sd of the residual
      
#     keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
#     output = combined_temp[combined_temp['YearMonth'].isin(keep)]
    return combined_temp, standard_dev

In [41]:
def conditional_factor_model(temp_conditional):
    '''
    @author: Robin Lam
    This function is for running the regression in the spirit of Grundy and Martin.
    Input:
        temp_conditional: dataframe which contains returns and factor exposures
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP
    Output:
        temp_conditional: same dataframe as the input, but with an extra column (residual)
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP, residual, alpha, mkt_beta, SMB_beta, HML_beta,
                       mkt_UP_beta, SMB_UP_beta, HML_UP_beta, t_mkt_beta, t_SMB_beta, t_HML_beta,
                       t_mkt_UP_beta, t_SMB_UP_beta, t_HML_UP_beta
    '''
    x = temp_conditional[['mkt_ret','SMB','HML','mkt_ret_UP','SMB_UP','HML_UP']]
    y = temp_conditional['ticker_excess_ret']

    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared
    
    # calculating the residuals for each regression model
    # temp_conditional has 36 rows (36 months)
    temp_conditional['residual'] = (y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] -
                                    x['mkt_ret_UP']*coef[4] - x['SMB_UP']*coef[5] - x['HML_UP']*coef[6] - coef[0])
    
    # adding coefficients to columns
    temp_conditional['alpha'] = coef[0]
    temp_conditional['mkt_beta'] = coef[1]
    temp_conditional['SMB_beta'] = coef[2]
    temp_conditional['HML_beta'] = coef[3]
    temp_conditional['mkt_UP_beta'] = coef[4]
    temp_conditional['SMB_UP_beta'] = coef[5]
    temp_conditional['HML_UP_beta'] = coef[6]
    temp_conditional['r_squared'] = r_squared
    
    # adding t-statistics to columns
    temp_conditional['t_alpha'] = t_test[0]
    temp_conditional['t_mkt_beta'] = t_test[1]
    temp_conditional['t_SMB_beta'] = t_test[2]
    temp_conditional['t_HML_beta'] = t_test[3]
    temp_conditional['t_mkt_UP_beta'] = t_test[4]
    temp_conditional['t_SMB_UP_beta'] = t_test[5]
    temp_conditional['t_HML_UP_beta'] = t_test[6]
  
    # get t-2, t-1 from "train"
    keep = [sorted(list(set(temp_conditional['YearMonth'].values)))[-1],sorted(list(set(temp_conditional['YearMonth'].values)))[-2]]
    output = temp_conditional[temp_conditional['YearMonth'].isin(keep)]
    
    return output

In [42]:
def all_regression(df):
    '''
    @author: Robin Lam, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        df: dataframe which contains ALL of the stocks' returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        output: same dataframe as the input, but with an extra column (residual)
        dict_coef: dictionary of coefficients from all the regression for ALL stocks. should have 3 betas.
        
    @Update: 10/16/2019: Wuding Li Change the output format of the function and the structure of the function
    '''
    output = pd.DataFrame()
    output_temp = pd.DataFrame()
    permno_list = list(set(df['PERMNO'].values.tolist()))
    sd_dic = {}
    for i in permno_list:
        temp_train = df[df['PERMNO'] == i]
        combined_temp, sd = regression_model(temp_train)    #there are 36 months in temp_train (individual ticker)
        output_temp = output_temp.append(combined_temp)
        keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
        output_keep = combined_temp[combined_temp['YearMonth'].isin(keep)]
        sd_dic[i] = sd
        output = output.append(output_keep)
    output = get_weight_Residual(output)   #this is also returned to merge with the test dataset for later!
        
    '''
    @author: Robin Lam
    Evaluating the reversal returns using a conditional factor model (Grundy and Martin)
    
        if excess returns on the RMRF, SMB, HML factors in month t are positive, use that factor;
        otherwise, factor = 0.
    '''
    df_conditional = format_Grundy(output_temp)
    output_conditional = pd.DataFrame()
    permno_list_conditional = list(set(df_conditional['PERMNO'].values.tolist()))
    for i in permno_list_conditional:
        temp_conditional = df_conditional[df_conditional['PERMNO'] == i]    #going through each ticker
        combined_ret = conditional_factor_model(temp_conditional)    #running Grundy regression
        output_conditional = output_conditional.append(combined_ret)
    output_conditional = get_weight_Residual(output_conditional)    #adding residual weight
    
    #get the exposure of residual weighted portfolio
    output['residual_port_mkt_beta'] = output['mkt_beta']*output['residual_weight']
    output['residual_port_SMB_beta'] = output['SMB_beta']*output['residual_weight']
    output['residual_port_HML_beta'] = output['HML_beta']*output['residual_weight']

    return sd_dic, output, output_conditional

In [43]:
def Grundy_Table(output_conditional, monthly_Grundy):
    '''
    ------------------------- NOT USED YET! -------------------------
    @author: Robin Lam
    This function gathers and outputs the things we need to replicate Table 1.
    Input:
        output_conditional: dataframe which was generated from the Grundy regression model.
    Output:
        monthly_Grundy: a dictionary which contains information for one month.
    '''
    #monthly_Grundy = {}
    output_conditional['monthly_ret'] = output_conditional['ticker_ret']*output_conditional['weight']
    sum_ret = output_conditional['monthly_ret'].sum()
    output_conditional['monthly_residual_ret'] = output_conditional['residual']*output_conditional['residual_weight']
    sum_residual_ret = output_conditional['monthly_residual_ret'].sum()
    #monthly_Grundy['sum_ret'] = sum_ret
    #monthly_Grundy['sum_residual_ret'] = sum_residual_ret
    
    output_conditional['residual_port_alpha'] = output_conditional['alpha']*output_conditional['residual_weight']
    output_conditional['residual_port_mkt_beta'] = output_conditional['mkt_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_SMB_beta'] = output_conditional['SMB_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_HML_beta'] = output_conditional['HML_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_mkt_UP_beta'] = output_conditional['mkt_UP_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_SMB_UP_beta'] = output_conditional['SMB_UP_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_HML_UP_beta'] = output_conditional['HML_UP_beta']*output_conditional['residual_weight']
    
    # Grundy table for conventional reversal strategy
    sum_port_alpha = output_conditional['port_alpha'].sum()
    sum_port_mkt_beta = output_conditional['port_mkt_beta'].sum()
    sum_port_SMB_beta = output_conditional['port_SMB_beta'].sum()
    sum_port_HML_beta = output_conditional['port_HML_beta'].sum()
    sum_port_mkt_UP_beta = output_conditional['port_mkt_UP_beta'].sum()
    sum_port_SMB_UP_beta = output_conditional['port_SMB_UP_beta'].sum()
    sum_port_HML_UP_beta = output_conditional['port_HML_UP_beta'].sum()
#     monthly_Grundy['sum_port_alpha'] = output_conditional['port_alpha'].sum()
#     monthly_Grundy['sum_port_mkt_beta'] = output_conditional['port_mkt_beta'].sum()
#     monthly_Grundy['sum_port_SMB_beta'] = output_conditional['port_SMB_beta'].sum()
#     monthly_Grundy['sum_port_HML_beta'] = output_conditional['port_HML_beta'].sum()
#     monthly_Grundy['sum_port_mkt_UP_beta'] = output_conditional['port_mkt_UP_beta'].sum()
#     monthly_Grundy['sum_port_SMB_UP_beta'] = output_conditional['port_SMB_UP_beta'].sum()
#     monthly_Grundy['sum_port_HML_UP_beta'] = output_conditional['port_HML_UP_beta'].sum()
    
    # Grundy table for residual reversal strategy
    sum_residual_port_alpha = output_conditional['residual_port_alpha'].sum()
    sum_residual_port_mkt_beta = output_conditional['residual_port_mkt_beta'].sum()
    sum_residual_port_SMB_beta = output_conditional['residual_port_SMB_beta'].sum()
    sum_residual_port_HML_beta = output_conditional['residual_port_HML_beta'].sum()
    sum_residual_port_mkt_UP_beta = output_conditional['residual_port_mkt_UP_beta'].sum()
    sum_residual_port_SMB_UP_beta = output_conditional['residual_port_SMB_UP_beta'].sum()
    sum_residual_port_HML_UP_beta = output_conditional['residual_port_HML_UP_beta'].sum()

#     monthly_Grundy['sum_residual_port_alpha'] = output_conditional['residual_port_alpha'].sum()
#     monthly_Grundy['sum_residual_port_mkt_beta'] = output_conditional['residual_port_mkt_beta'].sum()
#     monthly_Grundy['sum_residual_port_SMB_beta'] = output_conditional['residual_port_SMB_beta'].sum()
#     monthly_Grundy['sum_residual_port_HML_beta'] = output_conditional['residual_port_HML_beta'].sum()
#     monthly_Grundy['sum_residual_port_mkt_UP_beta'] = output_conditional['residual_port_mkt_UP_beta'].sum()
#     monthly_Grundy['sum_residual_port_SMB_UP_beta'] = output_conditional['residual_port_SMB_UP_beta'].sum()
#     monthly_Grundy['sum_residual_port_HML_UP_beta'] = output_conditional['residual_port_HML_UP_beta'].sum()
    
    monthly_Grundy = monthly_Grundy.append({'sum_ret': sum_ret, 'sum_residual_ret': sum_residual_ret,
                                           'sum_port_alpha': sum_port_alpha, 'sum_port_mkt_beta': sum_port_mkt_beta,
                                           'sum_port_SMB_beta': sum_port_SMB_beta, 'sum_port_HML_beta': sum_port_HML_beta,
                                           'sum_port_mkt_UP_beta': sum_port_mkt_UP_beta,
                                           'sum_port_SMB_UP_beta': sum_port_SMB_UP_beta,
                                           'sum_port_HML_UP_beta': sum_port_HML_UP_beta,
                                           'sum_residual_port_alpha': sum_residual_port_alpha,
                                           'sum_residual_port_mkt_beta': sum_residual_port_mkt_beta,
                                           'sum_residual_port_SMB_beta': sum_residual_port_SMB_beta,
                                           'sum_residual_port_HML_beta': sum_residual_port_HML_beta,
                                           'sum_residual_port_mkt_UP_beta': sum_residual_port_mkt_UP_beta,
                                           'sum_residual_port_SMB_UP_beta': sum_residual_port_SMB_UP_beta,
                                           'sum_residual_port_HML_UP_beta': sum_residual_port_HML_UP_beta},
                                           ignore_index=True)
                                            
    return monthly_Grundy

In [17]:
msf = pd.read_csv('msf.csv')
msf = data_cleaning(msf)

In [44]:
'''
Author: Zhikang Wang / Yulin Chen
MAIN FUNCTION.
Loop over the period from 1986 to generate rolling regression
'''

test_port = pd.DataFrame() #container for test df
'''
mkt_beta_winner_output = []
SMB_beta_winner_output = []
HML_beta_winner_output = []
mkt_beta_loser_output = []
SMB_beta_loser_output = []
HML_beta_loser_output = []
residual_mkt_beta_winner_output = []
residual_SMB_beta_winner_output = []
residual_HML_beta_winner_output = []
residual_mkt_beta_loser_output = []
residual_SMB_beta_loser_output = []
residual_HML_beta_loser_output = []
'''
output_conditional_output = pd.DataFrame()    #container for Grundy Table output

total_train_output = pd.DataFrame()

dates = []
ff = []
start = 1929

for i in range(710,944): #（7，328） （329，650） （651，944）
    if (i%12) != 0:
        month = i%12
        year = i // 12 + start
    if (i%12) == 0: 
        month = 12
        year = i // 12 + start -1

    ff_ret = format_FF()
    train,test = rolling_time_window(month, year, 37, 1, 0, msf)
    #train = get_equal_weight(train)
    train = get_weight(train)
    
    train = train.rename(columns = {"RET": "ticker_ret", "date": "YearMonth"})
    train = train[['YearMonth','PERMNO','ticker_ret','weight']]
    train["PERMNO"] = train["PERMNO"].astype(str)
    test["PERMNO"] = test["PERMNO"].astype(str)
    train["YearMonth"] = train["YearMonth"].astype(str)
    train['YearMonth'] = train['YearMonth'].apply(lambda x: x[:4] + x[5:7])
    # Merging the two dataset into one combined return dataset, which will be used as regression input
    combined_ret = pd.merge(ff_ret, train, how='right', on=['YearMonth'])
    # calculates the excess return of tickers by subtracting the returns by risk free rate
    combined_ret['ticker_excess_ret'] = combined_ret['ticker_ret'] - combined_ret['RF']
    #append dates
    month_list=["01","02","03","04","05","06","07","08","09","10","11","12"]
    month=month_list[month-1]
#     year = year + 3
    str_date = str(year)+str(month)
    dates.append(str_date)
        
    sd_dic, output, output_conditional = all_regression(combined_ret)
    output_conditional_output = output_conditional_output.append(output_conditional)
    total_train_output = total_train_output.append(output)
#     monthly_Grundy = pd.DataFrame()
    monthly_Grundy = pd.DataFrame(columns=['sum_ret','sum_residual_ret','sum_port_alpha','sum_port_mkt_beta',
                                          'sum_port_SMB_beta','sum_port_HML_beta','sum_port_mkt_UP_beta',
                                          'sum_port_SMB_UP_beta','sum_port_HML_UP_beta','sum_residual_port_alpha',
                                          'sum_residual_port_mkt_beta','sum_residual_port_SMB_beta',
                                          'sum_residual_port_HML_beta','sum_residual_port_mkt_UP_beta',
                                          'sum_residual_port_SMB_UP_beta','sum_residual_port_HML_UP_beta'])
    #monthly_Grundy = Grundy_Table(output_conditional, monthly_Grundy)
    #output_conditional_output = pd.concat([output_conditional_output, monthly_Grundy])
    
    sd_df = pd.DataFrame.from_dict(sd_dic, orient='index')
    sd_df = sd_df.reset_index()
    sd_df =  sd_df.rename(columns = {"index": "PERMNO", 0: 'std'})
    test = pd.merge(test,sd_df, how = 'left', on  =['PERMNO'])
    
    # Testing portfolio
    count = len(set(output['PERMNO'].values))
    output['test_conv_weight']=output['ticker_ret'].apply(lambda x: (x-output['ticker_ret'].mean())*(-1/count))
    output['test_red_weight'] = output['residual'].apply(lambda x: (x-output['residual'].mean())*(-1/count))
    output = output[['PERMNO','test_conv_weight','test_red_weight']]
    test = pd.merge(test,output,how = 'left',on=['PERMNO'])
    test_port = test_port.append(test)
    print(i)
    '''    
    mkt_beta_winner_output.append(mkt_beta_winner)
    SMB_beta_winner_output.append(SMB_beta_winner)
    HML_beta_winner_output.append(HML_beta_winner)
    mkt_beta_loser_output.append(mkt_beta_loser)
    SMB_beta_loser_output.append(SMB_beta_loser)
    HML_beta_loser_output.append(HML_beta_loser)
    
    residual_mkt_beta_winner_output.append(mkt_beta_winner_res)
    residual_SMB_beta_winner_output.append(SMB_beta_winner_res)
    residual_HML_beta_winner_output.append(HML_beta_winner_res)
    residual_mkt_beta_loser_output.append(mkt_beta_loser_res)
    residual_SMB_beta_loser_output.append(SMB_beta_loser_res)
    residual_HML_beta_loser_output.append(HML_beta_loser_res)
    '''

710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943


In [48]:
len(total_train_output)

154778

In [45]:
export_csv1 = total_train_output.to_csv ('train_output4.csv', index = None, header=True)
export_csv2 = test_port.to_csv ('test_output4.csv', index = None, header=True)
export_csv3 = output_conditional_output.to_csv ('conditional_output4.csv', index = None, header=True)